In [ ]:
import requests
import pandas as pd

SYMBOL = "KBANK"

session = requests.Session()
session.get(f"https://www.settrade.com/th/equities/quote/{SYMBOL}/financial-statement/full")


headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-TH,en;q=0.9,th-TH;q=0.8,th;q=0.7,en-GB;q=0.6,en-US;q=0.5',
    # 'cookie': 'visid_incap_2685215=l/JAPqmmR8KSk7rLhs3BzKvPSGYAAAAAQUIPAAAAAABALhVFFEmAe3vRYqQgqqQo; exp_history={"go_expid":"5AD93i4KR9-ZVNOhL9Vr2w-V2","msgt":"popup","count":1}; _cc_id=4d4cfaa1a003693b7c5cc45baf0913e9; __gads=ID=f9d48026689a99be:T=1716047793:RT=1716047793:S=ALNI_Ma_zJhrDoYFiXyfutD1XSHxPWA3Uw; __gpi=UID=00000e2256455e78:T=1716047793:RT=1716047793:S=ALNI_Mby0wdaHRd_h-eJjYJY5wLHS0TJAQ; __eoi=ID=2567fce2f1c28622:T=1716047793:RT=1716047793:S=AA-AfjaTaPE5D-KD9jKsPW8xY0gE; cto_bundle=44weE19pQlBrWWZRQUdySnclMkJmNXlkNkk4blI2MTgwTGNzSUZGSSUyRjRjdGRsUUZMZ1pWSDViOHIlMkY1c3lHc0NuZjBReDUlMkZVd21FT3piZkJ0MmE3bEEyRCUyQlVqYlFWbWpGVkQ0S1gxMlZUQTdIZXV3T05iN2VqbGZKWlNqRnloTm5jT3ltTnBCS1kzc3lxVWFUVEcxWFhHd2FnUjdCb1VMRWFzcmJKaG1SWm0lMkZqJTJGcE03c0UwJTJGYXRHU3FpUiUyQlhSRWlhMkxNd2o; cto_bidid=b-mYHF9lQlh6WHpHZHNta0hieDFuZ21FQ2dKQjdWRnR2RHNydllBQ2hkZEhOb1RxRzFhM1hGU29zbDgxcVdHOTFCQm1jJTJCb0ZaOVdGdjBPcCUyQnVmTGVVTUJHUmtCNDVkNTBOM1ZMdG9sVWg2OER1TFZDaENRNFElMkZjTjYwaFpGY0lsV3hjUUozZ2NUJTJGRG1KWWl5QlNqWWE5TVlKUSUzRCUzRA; visid_incap_2868064=9TnELvraSEyQ3rCeC/3tp6/imGYAAAAAQUIPAAAAAABQXj/9u+lgqIuVKksYLcer; _ga_W7C0TNQT63=GS1.1.1721377038.3.1.1721377082.0.0.0; charlot=6a8621f0-cdcf-45a6-b1c2-f81785e2d02f; _gcl_au=1.1.1685208698.1724255273; _gid=GA1.2.1052211754.1724255275; api_call_counter=5; SET_COOKIE_POLICY=20231111093657; nlbi_2685215=IC0uOhgeyBy9J7BzwZdY4QAAAAAqEQh9oBWiKDUUC927BZYM; _cbclose=1; _cbclose64035=1; route=0f35441e6abd8a06b9ca7c7f54fa2b5e; incap_ses_235_2685215=SjIpB/f0VXZxsG4/XeNCAyHexmYAAAAA4/zIE2DbjM+Hy9alp/2MXA==; landing_url=https://www.settrade.com/th/equities/quote/OR/overview?_gl=1%2a9iyx2w%2a_gcl_au%2aMTYwODA1NzI0NS4xNzIzNjQzNjg2%2a_ga%2aMTgyMzgyMTUxNS4xNzE1NTk1MjY5%2a_ga_ET2H60H2CB%2aMTcyNDI1NTIxMS4yNi4xLjE3MjQyNTU0ODguNDkuMC4w; _uid64035=DB23855A.7; _ctout64035=1; _gat_UA-29750633-11=1; _ga=GA1.1.1767658003.1716047789; _ga_W11E901KXL=GS1.1.1724324251.9.1.1724324280.31.0.0; _ga_6WS2P0P25V=GS1.1.1724324251.9.1.1724324280.31.0.0',
    'if-none-match': '"342c523b"',
    'priority': 'u=1, i',
    'referer': 'https://www.settrade.com/th/equities/quote/OR/financial-statement/full',
    'sec-ch-ua': '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
}

def get_sheet(period: str) -> pd.DataFrame:
    params = {
        'accountType': 'balance_sheet',
        'fsType': 'company',
        'period': period,
#         'language': 'th',
        'language': 'ENG',
    }

    response = session.get(
        'https://www.settrade.com/api/set/stock/OR/financialstatement',
        params=params,
        headers=headers,
    )
    response.raise_for_status()
    data = response.json()
    return pd.DataFrame(data["accounts"])